# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import os
import glob
import json
import re
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [36]:
def get_image_and_json_paths(src_path, num_files):
    image_paths = []
    json_paths = []

    for root, dirs, files in tqdm(os.walk(src_path), desc='Walking directories', unit=' dir'):
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            image_paths.extend(sorted(glob.glob(os.path.join(dir_path, '*.jpg'))[:num_files]))
            json_paths.extend(sorted(glob.glob(os.path.join(dir_path, '*.json'))[:num_files]))

    return image_paths, json_paths

def extract_data_from_json(json_paths):
    lesions, polygon_locations, box_locations = [], [], []
    for json_path in tqdm(json_paths, desc='Loading JSON', unit=' file'):
        try:
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.loads(re.sub(r'[\x00-\x1F\x7F-\x9F]', '', file.read()))
                metadata, labeling_info = json_data.get('metaData', None), json_data['labelingInfo']

                lesions.append(metadata.get('lesions', None))

                for entry in labeling_info:
                    if 'polygon' in entry:
                        polygon_locations.extend(entry['polygon'].get('location', None))
                    if 'box' in entry:
                        box_locations.extend(entry['box'].get('location', None))

        except Exception as e:
            print(json_path)
            print(e)

    return lesions, polygon_locations, box_locations

def preprocess_image(image, target_size):
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def create_dataset(image_paths, labels, buffer_size, batch_size, target_size):
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    image_dataset = image_dataset.map(lambda x: tf.io.read_file(x))
    image_dataset = image_dataset.map(lambda x: tf.image.decode_jpeg(x, channels=3))
    image_dataset = image_dataset.map(lambda x: preprocess_image(x, target_size))

    labels_dataset = tf.data.Dataset.from_tensor_slices(labels)

    dataset = tf.data.Dataset.zip((image_dataset, labels_dataset))

    dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.batch(batch_size)

    return dataset

# camera cat

In [37]:
src_path = '/content/drive/Shareddrives/반려묘/일반카메라'
num_files = 1000
buffer_size = 1000
batch_size = 64
target_size = (96, 96)

image_paths, json_paths = get_image_and_json_paths(src_path, num_files)
lesions, polygon_locations, box_locations = extract_data_from_json(json_paths)

onehot_encoder = OneHotEncoder(sparse_output=False)
lesions_array = np.array(lesions)
onehot_encoded = onehot_encoder.fit_transform(lesions_array.reshape(-1, 1))

Walking directories: 14 dir [00:00, 16.23 dir/s]
Loading JSON: 100%|██████████| 9375/9375 [48:17<00:00,  3.24 file/s] 


In [59]:
image_paths_shuffled, onehot_encoded_shuffled = shuffle(image_paths, onehot_encoded, random_state=42)

train_image_paths, remaining_image_paths, train_labels, remaining_labels = train_test_split(
    image_paths_shuffled, onehot_encoded_shuffled, train_size=0.8, random_state=42)

val_image_paths, test_image_paths, val_labels, test_labels = train_test_split(
    remaining_image_paths, remaining_labels, test_size=0.5, random_state=42)

train_dataset = create_dataset(train_image_paths, train_labels, buffer_size, batch_size, target_size)
val_dataset = create_dataset(val_image_paths, val_labels, buffer_size, batch_size, target_size)
test_dataset = create_dataset(test_image_paths, test_labels, buffer_size, batch_size, target_size)


train_size = len(train_image_paths)
val_size = len(val_image_paths)
test_size = len(test_image_paths)

print("Train dataset size:", train_size)
print("Validation dataset size:", val_size)
print("Test dataset size:", test_size)

Train dataset size: 7500
Validation dataset size: 937
Test dataset size: 938


In [60]:
labels = [label.numpy() for _, label in train_dataset]

labels = np.concatenate(labels, axis=0)

unique_labels, label_counts = np.unique(labels, return_counts=True, axis=0)

for label, count in zip(unique_labels, label_counts):
    print(f"Label: {label}, Count: {count}, Ratio: {count / len(labels)}")

Label: [0. 0. 0. 1.], Count: 4019, Ratio: 0.5358666666666667
Label: [0. 0. 1. 0.], Count: 1347, Ratio: 0.1796
Label: [0. 1. 0. 0.], Count: 1338, Ratio: 0.1784
Label: [1. 0. 0. 0.], Count: 796, Ratio: 0.10613333333333333


In [61]:
labels = [label.numpy() for _, label in val_dataset]

labels = np.concatenate(labels, axis=0)

unique_labels, label_counts = np.unique(labels, return_counts=True, axis=0)

for label, count in zip(unique_labels, label_counts):
    print(f"Label: {label}, Count: {count}, Ratio: {count / len(labels)}")

Label: [0. 0. 0. 1.], Count: 518, Ratio: 0.5528281750266809
Label: [0. 0. 1. 0.], Count: 161, Ratio: 0.1718249733191035
Label: [0. 1. 0. 0.], Count: 162, Ratio: 0.1728922091782284
Label: [1. 0. 0. 0.], Count: 96, Ratio: 0.10245464247598719


In [62]:
labels = [label.numpy() for _, label in test_dataset]

labels = np.concatenate(labels, axis=0)

unique_labels, label_counts = np.unique(labels, return_counts=True, axis=0)

for label, count in zip(unique_labels, label_counts):
    print(f"Label: {label}, Count: {count}, Ratio: {count / len(labels)}")

Label: [0. 0. 0. 1.], Count: 488, Ratio: 0.5202558635394456
Label: [0. 0. 1. 0.], Count: 167, Ratio: 0.17803837953091683
Label: [0. 1. 0. 0.], Count: 175, Ratio: 0.1865671641791045
Label: [1. 0. 0. 0.], Count: 108, Ratio: 0.11513859275053305


In [39]:
model = models.Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(units=4, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [63]:
history = model.fit(
    train_dataset,
    epochs=150,
    batch_size=64,
    validation_data=val_dataset,
)

Epoch 1/150
118/118 [==============================] - 18s 135ms/step - loss: 0.3837 - accuracy: 0.8531 - val_loss: 0.2361 - val_accuracy: 0.9274
Epoch 2/150
118/118 [==============================] - 18s 136ms/step - loss: 0.2665 - accuracy: 0.9021 - val_loss: 0.2272 - val_accuracy: 0.9242
Epoch 3/150
118/118 [==============================] - 18s 135ms/step - loss: 0.2096 - accuracy: 0.9289 - val_loss: 0.2400 - val_accuracy: 0.9136
Epoch 4/150
118/118 [==============================] - 18s 136ms/step - loss: 0.1818 - accuracy: 0.9385 - val_loss: 0.2282 - val_accuracy: 0.9157
Epoch 5/150
118/118 [==============================] - 18s 135ms/step - loss: 0.1622 - accuracy: 0.9445 - val_loss: 0.2498 - val_accuracy: 0.9146
Epoch 6/150
118/118 [==============================] - 18s 135ms/step - loss: 0.1348 - accuracy: 0.9549 - val_loss: 0.2846 - val_accuracy: 0.9039
Epoch 7/150
118/118 [==============================] - 18s 136ms/step - loss: 0.1230 - accuracy: 0.9601 - val_loss: 0.2716 -

In [64]:
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(onehot_encoded[train_size + val_size:], axis=1)
print(classification_report(y_true_classes, y_pred_classes))

15/15 [==============================] - 2s 4ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.65      0.79       938

    accuracy                           0.65       938
   macro avg       0.25      0.16      0.20       938
weighted avg       1.00      0.65      0.79       938



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('72.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import cv2
import numpy as np
from skimage.feature import greycomatrix
import h5py

class ImageFeatureExtractor:
    def __init__(self, target_size=(96, 96)):
        self.target_size = target_size

    def preprocess_image(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, self.target_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def extract_color_histogram(self, image):
        histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        histogram = cv2.normalize(histogram, histogram).flatten()
        return histogram

    def extract_hu_moments(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        moments = cv2.HuMoments(cv2.moments(gray_image)).flatten()
        return moments

    def extract_haralick_texture(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        texture = greycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True)
        haralick_features = np.mean(texture, axis=(0, 1, 2))
        return haralick_features

    def extract_features(self, image_path):
        image = self.preprocess_image(image_path)
        color_histogram = self.extract_color_histogram(image)
        hu_moments = self.extract_hu_moments(image)
        haralick_texture = self.extract_haralick_texture(image)
        global_features = np.concatenate([color_histogram, hu_moments, haralick_texture])
        return global_features

    def save_features_to_hdf5(self, features, lesions, output_path):
        with h5py.File(output_path, 'w') as f:
            f.create_dataset('features', data=features)
            f.create_dataset('lesions', data=lesions)

# HDF5 파일로부터 데이터 읽어오기
def load_data_from_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        features = f['features'][:]
        lesions = f['lesions'][:]
    return features, lesions

In [ ]:
feature_extractor = ImageFeatureExtractor(target_size=(96, 96))

features = []
for image_path in image_paths:
    image_features = feature_extractor.extract_features(image_path)
    features.append(image_features)

features = np.array(features)
lesions = np.array(lesions)

output_path = 'features.h5'
feature_extractor.save_features_to_hdf5(features, lesions, output_path)

In [ ]:
hdf5_file_path = 'path_to_your_hdf5_file.hdf5'

features, lesions = load_data_from_hdf5(hdf5_file_path)

X_train, X_test, y_train, y_test = train_test_split(features, lesions, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
history2 = model.fit(X_train, X_test, epochs=150, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
def combine_images_and_masks(image_list, mask_list):
    combined_images = []

    for i in tqdm(range(len(image_list)), desc='Combining Images and Masks'):
        image = image_list[i]
        mask = mask_list[i]

        mask_channel = np.zeros_like(image[:, :, 0], dtype=np.uint8)

        mask_channel[mask == 1] = 255

        combined_image = np.dstack((image, mask_channel))

        combined_images.append(combined_image)

    return combined_images

def one_hot_encode_labels(labels):
    label_binarizer = LabelBinarizer()
    encoded_labels = label_binarizer.fit_transform(labels)
    return np.array(encoded_labels), label_binarizer

def create_polygon_binary_masks(image_list, polygon_locations_list):
    binary_masks = []

    for image, polygon_locations in tqdm(zip(image_list, polygon_locations_list), desc='Generating Binary Masks', total=len(image_list)):
        mask = np.zeros(image.shape[:2], dtype=np.uint8)

        for poly_coords in polygon_locations:
            if poly_coords:
                poly_points = []
                i = 1
                while f'x{i}' in poly_coords and f'y{i}' in poly_coords:
                    x = poly_coords[f'x{i}']
                    y = poly_coords[f'y{i}']
                    poly_points.append([x, y])
                    i += 1
                if len(poly_points) > 0:
                    poly_points = np.array(poly_points, dtype=np.int32)
                    cv2.fillPoly(mask, [poly_points], 255)
            print(poly_coords)

        binary_masks.append(mask)

    return binary_masks

def visualize_masks(image_data, mask_data, selected_indices, figsize=(10, 5)):
    for idx in selected_indices:
        image = image_data[idx]
        mask_map = mask_data[idx]

        plt.figure(figsize=figsize)
        plt.subplot(1, mask_map.shape[2] + 1, 1)
        plt.imshow(image)
        plt.title('Original Image')

        for ch in range(mask_map.shape[2]):
            plt.subplot(1, mask_map.shape[2] + 1, ch + 2)
            plt.imshow(mask_map[:, :, ch], cmap='gray')
            plt.title(f'Channel {ch}')

        plt.tight_layout()
        plt.show()

# microscope cat

In [ ]:
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Concatenate, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mean_squared_error

def inception_module(input_layer, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), activation='relu')(input_layer)
    conv3x3_reduce = Conv2D(filters[1], (1, 1), activation='relu')(input_layer)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv2D(filters[3], (1, 1), activation='relu')(input_layer)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_layer)
    maxpool_conv = Conv2D(filters[5], (1, 1), activation='relu')(maxpool)
    inception_output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool_conv])
    return inception_output

# Input
input_shape = (128, 128, 3)
input_layer = Input(shape=input_shape)

# Inception block
inception_output = inception_module(input_layer, filters=[64, 128, 192, 32, 96, 64])
inception_output = inception_module(inception_output, filters=[64, 128, 192, 32, 96, 64])
# Add more inception modules if needed

# Primary Capsule layer
primary_capsules = Conv2D(32, (1, 1), activation='relu')(inception_output)

# Higher Capsule layers
# (Add imperative routing mechanism layers here)

# PReLU activation for routing
higher_capsules_prelu = PReLU()(higher_capsules)

# Flatten and Fully Connected layers
capsule_flatten = Flatten()(higher_capsules_prelu)  # Flatten higher capsules
output_layer = Dense(2, activation='softmax')(capsule_flatten)  # Two capsules: parasitized and uninfected

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with Adam optimizer and custom loss function
optimizer = Adam(learning_rate=0.007, beta_1=0.8)
loss_fn = custom_loss_function # Define the custom loss function as described in the paper
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()